In [2]:
import numpy as np
import math

In [3]:
def is_sym(A):
    '''
    input : numpy A 
    output : boolean if the Matrix A is symetric
    '''
    n,m = A.shape[0],A.shape[1]
    if (n != m):
        print("Not squared Matrix")
        return False
    for i in range(n//2):
        for j in range(n//2):
            if A[i,j] != A[j,i]:
                return False
    return True     

In [4]:
def determinant_naive(A):
    '''
    in : numpy A
    out : float 
    calculate the determinant of a matrix A 
    iterative method
    '''
    det = 0
    if (A.shape[0] != A.shape[1]):
        print("Not squared matrix")
        return 0
    n = A.shape[0]
    if (n == 2):
        det = A[0,0]*A[1,1] - A[0,1]*A[1,0]
        return det
    index = np.arange(n)
    for i in range(n):
        if (A[0,i] != 0):  
            det += pow(-1,i) * A[0,i] * determinant_naive(A[1:n,index !=i])
    return det
        

In [5]:
def permute_rows(A,i1,i2):
    n = A.shape[0]
    tmp = np.zeros(n)
    for j in range(n):
        tmp[j] = A[i1,j]
    A[i1,:]=A[i2,:]
    for j in range (n):
        A[i2,j] = tmp[j]
    return(A)

def determinant_gauss(A,show=False):
    '''
    in : numpy A
    out : float 
    calculate the determinant of a matrix A 
    gaussian elimination method
    show=True if you want to print the tringular matrix at the end
    '''
    A = A.astype(float)
    det = 0
    n,m = A.shape[0],A.shape[1]
    if (n !=m):
        print("Not squared Matrix")
        return 0
    number_permut = 0
    for j in range(n):
        row = n
        for i in range(j,n):
            if (A[i,j] != 0):
                row  = i
                break
        if (row == n):
            return 0
        if (row != j):
            permute_rows(A,j,row)
            number_permut +=1         
        for i in range(j+1,n):
            mult = A[i,j]/A[j,j]
            A[i,:] = A[i,:] - A[j,:] * mult
    if show:
        print(A)
    det = 1
    for i in range(n):
        det = det * A[i,i]
    return pow(-1,number_permut)*det
    

In [6]:
def is_def_pos_sylvester(A):
    '''
    input : numpy A a symetric matrix
    output : boolean if the matrix is positive definite
    Sylvester Criteria that check if all the minor determinant are positive
    '''
    n,m = A.shape[0],A.shape[1]
    if ( n != m):
        print("Not squared Matrix")
        return False
    for i in range(1,n+1):
        if (determinant_gauss(A[0:i,0:i]) <= 0):
            return False
    return True

In [7]:
def inverse_gauss(A):
    '''
    input : numpy A a squared matrix
    output : the inverse of A
    inverse by gaussian elimination
    '''
    A = A.astype(float)
    n,m = A.shape[0],A.shape[1]
    if (n !=m):
        print("Not squared Matrix")
        return np.zeros((n,n))
    inv = np.identity(n)
    for j in range(n):
        row = n
        for i in range(j,n):
            if (A[i,j] != 0):
                row  = i
                break
        if (row == n):
            return 0
        if (row != j):
            permute_rows(A,j,row)
            permute_rows(inv,j,row)        
        for i in range(j+1,n):
            mult = A[i,j]/A[j,j]
            A[i,:] = A[i,:] - A[j,:] * mult
            inv[i,:] = inv[i,:] - inv[j,:] * mult
    det = 1
    for i in range(n):
        det = det * A[i,i]
    if(det==0):
        print("Not inversible")
        return np.zeros((nn))
    for j in np.arange(n-1,0,-1):
        divisor = A[j,j]
        A[j,j] = A[j,j] / divisor # =1
        inv[j,:] = inv[j,:] / divisor #jusque la OK
        for i in np.arange(j-1,-1,-1):
            mult = A[i,j]
            A[i,:] = A[i,:] - mult *A[j,:]
            inv[i,:] = inv[i,:] - mult * inv[j,:]
    divisor = A[0,0]
    A[0,0] = A[0,0] / divisor
    inv[0,:] = inv[0,:] / divisor
    return(inv)
            
        

In [8]:
def cholesky(A):
    """
    Input : numpy A(n,n) (matrix symetric)
    Output : numpy L(n,n) The triangular lower matrix of the Cholesky Decomposition
    A = L * L.T
    """
    if not is_def_pos_sylvester(A):
        print("Not squared symetric matrix")
        return 0
    n = A.shape[0]
    L = np.zeros((n,n))
    L[0,0] = math.sqrt(A[0,0])
    for j in range(1,n):
        L[j,0] = A[0,j] / L[0,0]
    for i in range(1,n):
        L[i,i] = math.sqrt(A[i,i] -pow(L[i,0:i],2).sum())
        for j in range(i+1,n):
            L[j,i] = (A[i,j] - np.dot(L[i,0:i].T,L[j,0:i]))/L[i,i]
    return L

def determinant_cholesky(A):
    L = cholesky(A)
    determinant = 1
    for i in range(L.shape[0]):
        determinant = determinant * pow(L[i,i],2)
    return determinant

In [34]:
def power_iteration(A, num_simulations_max=pow(10,6), tol=pow(10,-6)):
    """
    input : A a squared matrix
    output : the biggest eigenvalor and the eigenvector associate
    power iteration method x = A*A*A*A*A*...*A(x)
    """
    x_n = np.random.rand(A.shape[1])

    for i in range(num_simulations_max):
        # calculate the matrix-by-vector product Ax
        x_n1 = np.dot(A, x_n)

        # calculate the norm L2 of vector
        x_n1_norm = np.linalg.norm(x_n1,2)

        # re normalize the vector
        x_n = x_n1 / x_n1_norm
        if (i>200 and np.linalg.norm(abs(x_n) - abs(x_old),2) < tol):
            break

        x_old = x_n
        if (i == num_simulations_max - 1):
            return([None,None])
    # Round at 3 decimal after coma
    eigenvalue = round(np.dot(np.dot(x_n.T,A),x_n)/np.dot(x_n.T,x_n),5)
    res = [eigenvalue, x_n]

    return res

def deflation(A, eigenvalue, v , w):
    """
    v is the eigenvector of A associate to the eigenvalue lambda
    v is the eigenvector of A.T associate to the eigenvalue lambda
    B = A - lambda * (v*w.T)/(w.T,v)
    """
    v = v.reshape(A.shape[0],1)
    w = w.reshape(A.shape[0],1)
    B = A - eigenvalue * np.dot(v,w.T) / np.dot(w.T,v)
    return B

In [35]:
def eigen(A):
    '''
    input : numpy squared matrix A
    out : eigenvalues and eigevectors of the matrix if A is diagonalizable
    '''
    B = A
    if (B.shape[0] != B.shape[1]):
        print("Not squared Matrix")
        return (0,0)
    eigenvectors = []
    eigenvalues = []
    for i in range(B.shape[0]):
        tmp = power_iteration(B)
        if (tmp[0] == None) :
            print("Matrix is not diagonalizable")
            break
        eigenvalues.append(tmp[0])
        eigenvectors.append(tmp[1])
        tmp2 = power_iteration(B.T)#To apply deflation method we need to have eigenvector of the transpose matrix
        B = deflation(B,tmp[0],tmp[1],tmp2[1])
    return eigenvalues,eigenvectors

In [36]:
def eigen_matrix(A):
    """
    input : numpy squared matrix A
    out : the transition matrix P if A is diagonalizable such that A = inv(P)*D*P
    """
    eigenvalues,eigenvectors = eigen(A)
    n = A.shape[0]
    P = np.zeros((n,n))
    for i in range(n):
        P[:,i] = eigenvectors[i]
    return P

In [37]:
T = np.array([[0.941088,0.056529,0.001942,0.000101,0.0000998,0.0000996,0.0000995,0.0000408],
[0.032701,0.895942,0.06195,0.002989,0.002694,0.002241,0.000312,0.001171],
[0.022173,0.033878,0.891907,0.040948,0.004598,0.004589,0.000313,0.001593],
[0.017766,0.018082,0.051682,0.881276,0.015007,0.006292,0.006272,0.003623],
[0.000973,0.007335,0.013373,0.067571,0.8222,0.066269,0.007225,0.015053],
[0.000337,0.000667,0.013177,0.019136,0.060705,0.808061,0.05517,0.042747],
[0.000224,0.000399,0.008528,0.013169,0.032274,0.097095,0.75089,0.097422],
[0,0,0,0,0,0,0,1]])

In [38]:
print(T)

[[9.41088e-01 5.65290e-02 1.94200e-03 1.01000e-04 9.98000e-05 9.96000e-05
  9.95000e-05 4.08000e-05]
 [3.27010e-02 8.95942e-01 6.19500e-02 2.98900e-03 2.69400e-03 2.24100e-03
  3.12000e-04 1.17100e-03]
 [2.21730e-02 3.38780e-02 8.91907e-01 4.09480e-02 4.59800e-03 4.58900e-03
  3.13000e-04 1.59300e-03]
 [1.77660e-02 1.80820e-02 5.16820e-02 8.81276e-01 1.50070e-02 6.29200e-03
  6.27200e-03 3.62300e-03]
 [9.73000e-04 7.33500e-03 1.33730e-02 6.75710e-02 8.22200e-01 6.62690e-02
  7.22500e-03 1.50530e-02]
 [3.37000e-04 6.67000e-04 1.31770e-02 1.91360e-02 6.07050e-02 8.08061e-01
  5.51700e-02 4.27470e-02]
 [2.24000e-04 3.99000e-04 8.52800e-03 1.31690e-02 3.22740e-02 9.70950e-02
  7.50890e-01 9.74220e-02]
 [0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00
  0.00000e+00 1.00000e+00]]


In [39]:
eigvals,_ = eigen(T)

In [40]:
D = np.diag(eigvals)
print("D \n :",D)
L = np.diag(np.log(eigvals))
print("L \n :",L)
print(L)

D 
 : [[1.      0.      0.      0.      0.      0.      0.      0.     ]
 [0.      0.99587 0.      0.      0.      0.      0.      0.     ]
 [0.      0.      0.93461 0.      0.      0.      0.      0.     ]
 [0.      0.      0.      0.89277 0.      0.      0.      0.     ]
 [0.      0.      0.      0.      0.86453 0.      0.      0.     ]
 [0.      0.      0.      0.      0.      0.83027 0.      0.     ]
 [0.      0.      0.      0.      0.      0.      0.77677 0.     ]
 [0.      0.      0.      0.      0.      0.      0.      0.69656]]
L 
 : [[ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.         -0.00413855  0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.         -0.06762595  0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.         -0.11342629  0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.         -0.14556927  0.
   0.        

In [22]:
P=eigen_matrix(T)
print(P)

[[ 4.01084456e-01  5.25212996e-01 -4.54678760e-01  4.56236536e-01
  -5.03519833e-01 -2.79336087e-01 -4.17538563e-03 -1.87742199e-04]
 [ 3.92020123e-01  4.79497760e-01  4.19983295e-02 -3.83180654e-01
   6.83638283e-01  5.63578760e-01  1.19440566e-02  2.11216556e-03]
 [ 3.86422152e-01  4.51273849e-01  2.08953179e-01 -2.56907215e-01
  -8.55417228e-02 -4.62167167e-01 -8.39115682e-03 -1.25668497e-02]
 [ 3.77754635e-01  4.07551731e-01  3.13649452e-01 -3.89957770e-02
  -2.88583320e-01  4.03680513e-01  4.99989561e-02  3.30058082e-02]
 [ 3.49285388e-01  2.63979023e-01  5.21606272e-01  3.93771181e-01
   1.48179416e-01  1.40904451e-01 -7.26963570e-01 -2.50708850e-01]
 [ 3.25771553e-01  1.45385635e-01  4.83562515e-01  5.01532078e-01
   3.08360964e-01 -3.24527077e-01  3.86222436e-01  5.32067102e-01]
 [ 3.10051315e-01  6.60994165e-02  3.78893318e-01  4.13410999e-01
   2.67329633e-01 -3.20346682e-01  5.65354575e-01 -8.07959074e-01]
 [ 2.63005005e-01 -1.71159768e-01  4.37197168e-07  1.12914835e-11
  -

In [26]:
T_2 = np.dot(P,np.dot(D,inverse_gauss(P)))
print(T_2)

[[ 9.41537387e-01  5.69667473e-02  2.29849030e-03  2.55508830e-04
   1.52323350e-04  1.50053629e-04  1.18234812e-04 -2.87883238e-03]
 [ 3.31337877e-02  8.96314186e-01  6.22878066e-02  3.17720051e-03
   2.74426789e-03  2.28626347e-03  3.29510747e-04 -1.71360690e-03]
 [ 2.25839450e-02  3.42429831e-02  8.92242425e-01  4.11311196e-02
   4.64158272e-03  4.62660618e-03  3.27393066e-04 -1.26285242e-03]
 [ 1.81537217e-02  1.84398623e-02  5.19946295e-02  8.81439769e-01
   1.50488996e-02  6.32935879e-03  6.28619539e-03  8.02636257e-04]
 [ 1.29540637e-03  7.64602314e-03  1.36170884e-02  6.76671001e-02
   8.22245488e-01  6.63111589e-02  7.24010906e-03  1.23432169e-02]
 [ 6.01412365e-04  9.27073237e-04  1.33701491e-02  1.91999115e-02
   6.07419098e-02  8.08103149e-01  5.51852687e-02  4.01315177e-02]
 [ 4.46948736e-04  6.17742073e-04  8.69093153e-03  1.32237622e-02
   3.23045849e-02  9.71289658e-02  7.50905013e-01  9.48724208e-02]
 [ 9.63352276e-05  8.82411701e-05  7.64317122e-05  3.83057040e-05
   

In [25]:
print(np.linalg.norm(T-T_2))

0.007851639222354604
